In [ ]:
import numpy as np 
import pandas as pd 
import os
from pathlib import Path
import cv2
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt

In [ ]:
BASE_PATH = '../input/siim-covid19-detection/'
train_study = pd.read_csv(BASE_PATH + 'train_study_level.csv')
train_study.head()

In [ ]:
train_image = pd.read_csv(BASE_PATH + 'train_image_level.csv')
train_image.head()

In [ ]:
train_image['split_label'] = train_image.label.apply(lambda x: [x.split()[offs:offs+6] for offs in range(0, len(x.split()), 6)])
train_image.head()

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

def plot_img(img, size=(7, 7), is_rgb=True, title="", cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

def plot_imgs(imgs, cols=4, size=7, is_rgb=True, title="", cmap='rainbow', img_size=(500,500)):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(cols*size, rows*size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

In [ ]:
from fastai.vision.all import *
from fastai.medical.imaging import *
dataset_path = Path('../input/siim-covid19-detection')
def image_path(row):
    study_path = dataset_path/'train'/row.StudyInstanceUID
    for i in get_dicom_files(study_path):
        if row.id.split('_')[0] == i.stem: return i 

train_image['image_path'] = train_image.apply(image_path, axis=1)
train_image.head()

In [ ]:
imgs = []
image_paths = train_image['image_path'].values
color_mean=(104,11,12)
# map label_id to specify color
thickness = 5
scale = 5


for i in range(8):
    image_path = random.choice(image_paths)
    img = dicom2array(path=image_path)
    img = cv2.resize(img, None, fx=1/scale, fy=1/scale)
    

    img = np.stack([img, img, img], axis=-1)
    for i in train_image.loc[train_image['image_path'] == image_path].split_label.values[0]:
        if i[0] == 'opacity':
            img = cv2.rectangle(img,
                                (int(float(i[2])/scale), int(float(i[3])/scale)),
                                (int(float(i[4])/scale), int(float(i[5])/scale)),
                                [255,0,0], thickness)
    
    img = cv2.resize(img, (500,500))
    
    img = img - color_mean
    imgs.append(img)
    
plot_imgs(imgs, cmap="rainbow")
